In [31]:
from selenium import webdriver
import pandas as pd
import re


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
import time

chrome_driver_path = r"D:\chromedriver-win64\chromedriver-win64\chromedriver.exe"
driver = webdriver.Chrome(service=Service(chrome_driver_path))
url = "https://cn.bing.com/search?q=%E5%B9%BF%E5%B7%9E%E5%B8%82%E6%96%B0%E6%B8%AF%E4%B8%AD%E8%B7%AF%20%E6%98%AF%E4%BB%80%E4%B9%88%E5%8C%BA&qs=n&form=QBRE&=%E5%9C%A8%7B0%7D%E4%B8%AD%E6%90%9C%E7%B4%A2%7B1%7D&=%E6%90%9C%E7%B4%A2%7B0%7D%E7%9A%84%E5%B7%A5%E4%BD%9C&=%25e%E7%AE%A1%E7%90%86%E6%90%9C%E7%B4%A2%E5%8E%86%E5%8F%B2%E8%AE%B0%E5%BD%95%25E&sp=-1&lq=0&pq=%E5%B9%BF%E5%B7%9E%E5%B8%82%E6%96%B0%E6%B8%AF%E4%B8%AD%E8%B7%AF%20%E6%98%AF%E4%BB%80%E4%B9%88%E5%8C%BA&sc=0-12&sk=&cvid=972BFCC3DFB747A587A20B970FDD5F49&ghsh=0&ghacc=0&ghpl="
driver.get(url)

In [32]:
# df = pd.read_excel(r'C:\Users\gengly\Desktop\华南0621.xlsx',sheet_name='医院基础信息核对')
# df = df[(df['是否核对']=='Y') & df['区'].isna()]

In [33]:
#df = pd.read_excel('爬地区.xlsx')

In [34]:
#df = pd.read_excel('爬到的地址.xlsx')

In [35]:
df = pd.read_excel(r'C:\Users\cxy\OneDrive\桌面\回国\实习\8-7\Python医院基础信息核对-CHENXY - 省市区.xlsx',sheet_name='基础信息核对 (2)')

In [36]:
def pre_matching(address):
    pattern = re.compile(r'([\u4e00-\u9fa5]+省|[\u4e00-\u9fa5]+市|[\u4e00-\u9fa5]+区|[\u4e00-\u9fa5]+县)')
    matches = pattern.findall(address)
    if matches:
        if all(excluded not in matches[-1] for excluded in ['广西壮族自治区', '广西', '广东', '海南']):
            return matches
        

In [37]:
def extract_region(address):
    if address is not None and not (isinstance(address, float)):
        # 优先匹配市级信息，如果有多个市，提取最后一个市
        cities = re.findall(r'([^市]+?市)', address)
        if cities:
            return(cities[-1])
        # 如果没有市级信息，再匹配区或县信息
        district = re.search(r'([^市]+?(区|县))', address)
        if district:
            return (district[0])
        #如果没有区县信息，再匹配乡镇
        town = re.search(r'([^市]+?(乡|镇))', address)
        if town:
            return (town[0])
    return None

In [38]:
for i in range(len(df)):
    address = df['地址'].iloc[i]
    matches=None
    if address is not None and not (isinstance(address, float)):
        #预匹配
        pre_match = pre_matching(address)
        if pre_match:
            if (pre_match[-1] != df['市'].iloc[i]) & (pre_match[-1] != df['省'].iloc[i]):
                df['区'].iloc[i] = pre_match[-1]
                continue
        
        #清洗地址
        pattern = re.compile(r"^\D+")
        result = pattern.search(address).group(0)

        #增加重做机制
        retries=5
        for _ in range(retries):
            try:
                #定位搜索框
                search_box = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.ID, 'sb_form_q'))
                )
                search_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.ID, 'sb_form_go'))
                )
                continue
            except Exception as e:
                print(f"Exception encountered: {e}. Retrying...")
        
        #清空搜索框
        search_box.send_keys(Keys.CONTROL + 'a')
        search_box.send_keys(Keys.BACKSPACE)
        
        #输入地址
        address_clean = result+' 属于什么区'
        search_box.send_keys(address_clean)        
        
        #点击搜索按钮                
        search_button.click()
        
        #等待搜索，提取区信息
        wait = WebDriverWait(driver, 1)
        try:
            for _ in range(3):
                try:
                    div_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'b_focusTextLarge')))
                    if div_element.text: 
                        matches = extract_region(div_element.text)
                        break
                except:
                    try:
                        div_element =  wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'b_focusTextMedium')))
                        if div_element.text: 
                            matches = extract_region(div_element.text)
                            break
                    except:
                        try:
                            p_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'b_paractl')))
                            if p_element.text:
                                keywords = ['区','县']
                                for element in pre_matching(p_element.text):
                                    if any(keyword in element for keyword in keywords):
                                        matches = element
                                        break 
                                if matches:
                                    break
                        except:
                            continue
        except:
            df['区'].iloc[i] = '查询不到'
            continue

        # for _ in range(3):
        #     if div_element.text: 
        #         matches = extract_region(div_element.text)
        #         break
        #     else: 
        #         if p_element.text:
        #             keywords = ['区','县']
        #             for element in pre_matching(p_element.text):
        #                 if any(keyword in element for keyword in keywords):
        #                     matches = element
        #                     break 
        #             if matches:
        #                 break
        
        #写入dataframe
        df['区'].iloc[i] = matches


C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = matches
C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = pre_match[-1]
C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = matches
C:\Users\cxy\AppData

C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = matches
C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = matches
C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = matches
C:\Users\cxy\AppData\Loca

C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = matches
C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = pre_match[-1]
C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = pre_match[-1]
C:\Users\cxy\Ap

C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = matches
C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = matches
C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = pre_match[-1]
C:\Users\cxy\AppData

C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = matches
C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = pre_match[-1]
C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = pre_match[-1]
C:\Users\cxy\Ap

C:\Users\cxy\AppData\Local\Temp\ipykernel_16040\1732102770.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['区'].iloc[i] = matches


In [39]:
i

71

In [39]:
df.to_excel(r'C:\Users\cxy\OneDrive\桌面\回国\实习\8-7\Python地址第三次.xlsx',index=False)

In [ ]:
pre_match = pre_matching('广州市大坦沙岛双桥')


In [ ]:
if (pre_match[-1] != df['市'].iloc[0]) & (pre_match[-1] != df['省'].iloc[0]):
    print(pre_match[-1])

In [ ]:
df['市'].iloc[0]